In [1]:
import os
import pandas as pd
import numpy as np

In [5]:
os.getcwd()

'C:\\Users\\girel\\OneDrive\\Documentos\\python\\conditional_independence_discrete_graphical_model'

In [6]:
mpi2023 = pd.read_csv(os.getcwd() + "\\data\\MPI2023_microdata.csv")

In [7]:
mpi2023.head()

,hhid,hh_d_ni_noasis,hh_d_esc_retardada,hh_d_logro_min,hh_d_destotalmax,hh_d_subocup_max,hh_d_10a17_ocup,hh_d_no_afil,hh_d_jubi_pens,hh_d_materialidad,...,upm,nvivi,nhoga,dpto,area,pobrezai,pobnopoi,ing_tcm,tcm,estgeo
0,00014_003_1,0,1,0,1,0,0,0,0,0,...,14,3,1,Asuncion,1,3,0,0.0,0,Asuncion
1,00014_020_1,0,0,0,0,0,0,1,0,0,...,14,20,1,Asuncion,1,3,0,0.0,0,Asuncion
2,00014_023_1,0,0,0,0,0,0,1,0,0,...,14,23,1,Asuncion,1,3,0,0.0,0,Asuncion
3,00014_024_1,0,0,0,0,0,0,0,0,0,...,14,24,1,Asuncion,1,3,0,0.0,0,Asuncion
4,00014_025_1,0,0,0,0,0,0,0,0,0,...,14,25,1,Asuncion,1,3,0,0.0,0,Asuncion


In [8]:
dimensions_indicators = {
    "ed" : ["hh_d_ni_noasis","hh_d_esc_retardada","hh_d_logro_min"],
    "em" : ["hh_d_destotalmax","hh_d_subocup_max","hh_d_10a17_ocup","hh_d_no_afil","hh_d_jubi_pens"],
    "vi" : ["hh_d_materialidad","hh_d_hacinamiento","hh_d_sin_basur"],
    "sa" : ["hh_d_sin_salud","hh_d_agua_mejor","hh_d_san_mejor","hh_d_combus"]
}

In [9]:
def calculate_weights(mpi_indicators):
  """
  Parameters:
    mpi_indicators: dictionary

  Computation of each dimension and indicators weigths following capability approach criterion

  Returns two dictionaries: one with dimensions weigths and another with indicators weights
  """
  dim_weights, indic_weights = {}, {}
  for key in mpi_indicators.keys():
    weight = 1/len(mpi_indicators.keys())
    dim_weights[key] = weight
    for value in mpi_indicators[key]:
      indic_weights[value] = weight / len(mpi_indicators[key])
  return dim_weights, indic_weights

In [10]:
def deprivation_score(mpi_indicators,data):
  dimensions_weights, indicators_weights = calculate_weights(mpi_indicators)
  indicators_ = list(indicators_weights.keys())
  mpi_data = data[indicators_]
  for indicator in mpi_data.columns:
    mpi_data[indicator] *= indicators_weights[indicator]
  score = mpi_data.sum(axis=1)
  return score

In [11]:
def censored_deprivation_score(deprivation_score, k):
  censored_deprivation_score = np.where(deprivation_score >= k, deprivation_score, 0)
  return censored_deprivation_score

In [12]:
c_k = censored_deprivation_score(deprivation_score(dimensions_indicators, mpi2023), 26/100)

C:\Users\girel\AppData\Local\Temp\ipykernel_32524\543157394.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mpi_data[indicator] *= indicators_weights[indicator]


In [13]:
mpi2023['mpi_poor'] =  np.where(c_k > 0, 1, 0)

In [17]:
nested_indicators = list(dimensions_indicators.values())

indicators = []
for i in nested_indicators:
    indicators.extend(i)

In [18]:
from sklearn import model_selection

In [19]:
X = mpi2023[indicators]
Y = mpi2023['pobnopoi']

Xtrain, Xtest, ytrain, ytest = model_selection.train_test_split(X,Y,train_size = 0.8, random_state=912)

In [ ]:
from discrete_gm_nonpos import direct_ci_model
ci = direct_ci_model(c=np.array(0))
Ytrain = ytrain.to_numpy().reshape(-1, 1)
Xtrain = Xtrain.to_numpy()
ci.learn(Xtrain>0, Ytrain>0)

In [ ]:
direct_ci_pred = ci.predict(Xtest>0)

In [32]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=500)
random_forest = rf.fit(Xtrain, ytrain)

In [33]:
from sklearn.metrics import confusion_matrix

def leakage(ytrue, ypred):
    tn, fp, fn, tp = confusion_matrix(ytrue, ypred).ravel()
    metric = fp/(fn+tp) 
    if metric > 1: metric = 1 
    return metric

def undercoverage(ytrue,ypred):
    tn, fp, fn, tp = confusion_matrix(ytrue, ypred).ravel()
    metric = fn/(fn+tp) 
    if metric > 1: metric = 1
    return metric
    
def evaluate_thresholds(probabilities, true_labels, thresholds=None):
    if thresholds is None:
        thresholds = np.arange(0, 1.01, 0.01)

    results = {
        'thresholds': [],
        'leakage': [],
        'undercoverage': []
    }

    for threshold in thresholds:
        predictions = (probabilities[:, 1] >= threshold).astype(int)
        leakage_error = leakage(true_labels, predictions)
        undercoverage_error = undercoverage(true_labels, predictions)

        results['thresholds'].append(threshold)
        results['leakage'].append(leakage_error)
        results['undercoverage'].append(undercoverage_error)
    
    return results
        

In [34]:
# Assuming you have probabilities and true_labels
results_rf = evaluate_thresholds(random_forest.predict_proba(Xtest), ytest)

In [39]:
Ytrain

array([[0],
       [1],
       [1],
       ...,
       [0],
       [0],
       [0]], dtype=int64)